# Model-based Collaborative Filtering

## Imports

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
import os
print(os.listdir("./data"))

['movie.csv', 'rating.csv']


## Data Preprocessing

In [3]:
movies = pd.read_csv('./data/movie.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies['year'] = (movies.title.str.extract('(\(\d\d\d\d\))', expand=False).str.extract('(\d\d\d\d)', expand=False))
movies['title'] = (movies.title.str.replace('(\(\d\d\d\d\))', '').apply(lambda x: x.strip()))
movies['genres'] = movies.genres.str.split('|')

movies.head()

/home/mgh/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [5]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
 3   year     27256 non-null  object
dtypes: int64(1), object(3)
memory usage: 852.6+ KB


In [6]:
ratings = pd.read_csv('./data/rating.csv', usecols=['userId', 'movieId', 'rating'],
                     dtype={'userId':np.int32, 'movieId':np.int32, 'rating':np.float32})
ratings.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


Due to huge memory usage, we can further decrease our data by multiplying these columns with 2 to make everthing int and then convert back to np.int8.

In [7]:
ratings['rating'] = ratings['rating'] * 2
ratings['rating'] = ratings['rating'].astype(np.int8)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   userId   int32
 1   movieId  int32
 2   rating   int8 
dtypes: int32(2), int8(1)
memory usage: 171.7 MB


In [8]:
ratings.head()

,userId,movieId,rating
0,1,2,7
1,1,29,7
2,1,32,7
3,1,47,7
4,1,50,7


## SVD Matrix Factorization

With these systems you build a model from user ratings, and then make recommendations based on that model. This offers a speed and scalability that's not available when you're forced to refer back to the entire dataset to make a prediction. We are going to see something called a utility matrix.

Utility matrix is also known as user item matrix. These matrices contain values for each user, each item, and the rating each user gave to each item. Another thing to note is that utility matrices are sparse because every user does not review every item. Actually, only a few users provide reviews for a few items. So in these matrices, we are likely to see mostly null values. Before explaining the truncated version, let's see the regular singular value decomposition or SVD.

SVD is a linear algebra method that you can use to decompose a utility matrix into three compressed matrices. It's useful for building a model-based recommender because we can use these compressed matrices to make recommendations without having to refer back to the complete and entire dataset. With SVD, we uncover latent variables. These are inferred variables that are present within and affect the behavior of a dataset. Although these variables are present and influential within a dataset, they're not directly observable. Now let's look at the anatomy of SVD.

Utility Matrix = U x S x V

We see three resultant matrices, U, S, and V. U is the left orthogonal matrix, and it holds the important, non-redundant information about users. On the right, we see matrix V. That's the right orthogonal matrix. It holds important, non-redundant information on items. In the middle, we see S, the diagonal matrix. This contains all of the information about the decomposition processes performed during the compression.

We want to use the similarities between users, to decide which movies to recommend, so we can use truncated SVD to compress all of the user ratings down to just small number of latent variables. These variables are going to capture most of the information that was stored in user columns previously. They represent a generalized view of users' tastes and preferences. The first thing we will do is to transpose our matrix, so that movies are represented by rows, and users are represented by columns. Then we'll use SVD to compress this matrix. All of the individual movie names will be retained along the rows. But the users will have been compressed down to number synthetic components which we will choose, that represent a generalized view of users' tastes.

In [10]:
# Due to problems with pandas, we can't use pivot_table with our all data as it throws MemoryError.
# Therefore, for this part we will work with a sample data
sample_ratings = ratings.sample(n=100000, random_state=20)

# Creating our sparse matrix and fill NA's with 0 to avoid high memory usage.
pivot = pd.pivot_table(sample_ratings, values='rating', index='userId', columns='movieId', fill_value=0)
pivot.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,120610,120819,121235,123947,125916,126420,127622,128151,129659,130490
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
pivot = pivot.astype(np.int8)
pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52242 entries, 1 to 138493
Columns: 8433 entries, 1 to 130490
dtypes: int8(8433)
memory usage: 420.5 MB


In [12]:
from sklearn.decomposition import TruncatedSVD

X = pivot.T
SVD = TruncatedSVD(n_components=500, random_state=20)
SVD_matrix = SVD.fit_transform(X)

Let's see how much of these 500 variables cover the whole data

In [13]:
SVD.explained_variance_ratio_.sum()

0.522339206899319

We see that it covers about 52% of our whole data.

### Generating a Correlation Matrix

In [14]:
# We'll calculate the Pearson r correlation coefficient, 
# for every movie pair in the resultant matrix. With correlation being 
# based on similarities between user preferences.

corr_mat = np.corrcoef(SVD_matrix)
corr_mat.shape

(8433, 8433)

### Isolating One Movie From the Correlation Matrix

Let's stick with Pulp Fiction choice

In [16]:
rand_movie = 296

corr_pulp_fiction = corr_mat[rand_movie]

# Recommending a Highly Correlated Movie.
# We will get different results due to decompression with svd
idx = X[(corr_pulp_fiction < 1.0) & (corr_pulp_fiction > 0.5)].index
movies.loc[idx+1, 'title']

movieId
275    Miami Rhapsody
Name: title, dtype: object